In [1]:
import yaml

In [2]:
import csv

In [168]:
PAGES = {
    "Arkansas": "675-688",
    "Alabama": "662-660",
    "California": "690-691",
    "Florida": "693-696",
    "Georgia": "700-723",
    "Kansas": "725-726",
    "Kentucky": "730-738",
    "Louisiana": "743-757",
    "Maryland": "759-764",
    "Minnesota": "765",
    "Mississippi": "771-789",
    "Missouri": "797-815",
    "North_Carolina": "818-826",
    "Pennsylvania": "827-828",
    "South_Carolina": "831-835",
    "Tennessee": "845-878",
    "Texas": "880-881",
    "Virginia": "894-965",
    "West_Virginia": "970-980",
    "Territories": "984-989",
    "Miscellaneous": "990-991"
}

In [97]:
import datetime
import re
import os
from ruamel.yaml import YAML
yaml = YAML(typ='safe')
yaml.default_flow_style = False
yaml.representer.ignore_aliases = lambda x: True

def iter_files():
    outdir = 'engagements'
    for filename in os.listdir(outdir):
        fn = os.path.join(outdir, filename)
        state, ext = os.path.splitext(fn)
        if ext == '.yml':
            with open(fn, 'r') as fp:
                data = yaml.load(fp)
            yield (fn, data)


In [157]:
from ruamel.yaml import YAML
yaml = YAML(typ='safe')
yaml.default_flow_style = False
yaml.representer.ignore_aliases = lambda x: True

In [199]:
from slugify import slugify
def make_id(state, start_date, end_date, nature_location, num):
    if end_date == start_date:
        date_str = start_date.strftime("%Y-%m-%d")
    else:
        date_str = '-'.join(
            (start_date.strftime("%Y-%m-%d"),
             end_date.strftime("%Y-%m-%d")))
    location_slug = slugify(nature_location, word_boundary=True)
    num = f'-{num}' if num else ''
    return "{}/{}-{}{}".format(state, date_str, location_slug, num)
  

In [202]:
import shutil
from slugify import slugify

foo = set()
for filename in os.listdir(outdir):
    fn = os.path.join(outdir, filename)
    state, ext = os.path.splitext(fn)
    if ext == '.yml':
        shutil.copy(fn, fn + '.bak')
        with open(fn, 'r') as fp:
            data = yaml.load(fp)
        for row in data:
            new_id = make_id(re.sub(" ", "_", row['state']), 
                             row['start_date'],
                             row["end_date"],
                             row['nature_location'],
                             row.get('num'))
            if new_id in foo:
                print(new_id, " is a duplicate")
            else:
                foo.add(new_id)
            row['slug'] = new_id
        with open(fn, 'w') as fp:
            yaml.dump(data, fp)            